# Libs

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import regex as re

In [2]:
data_path = "D:/DANG NHI/Cao học/Khai thác văn bản/Đồ án cuối kỳ/data"

In [3]:
MAX_LEN = 512
OVERLAP = 100

In [6]:
def preprocess_phrase1(input_text):
    s = input_text
    s = re.sub('CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM', '\n', s)
    s = re.sub('Độc lập - Tự do - Hạnh phúc', '\n', s)
    s = re.sub('[- ]{3,}', ' ', s)
    s = re.sub('(?<=\n)[\n ]+', ' ', s)
    return s

def count_words(input_text):
    return len(input_text.split())

def markdown_chunking(input_text, max_len=512, overlap=100):
    sentences = input_text.split('\n')
    lst_counts = [count_words(i) for i in sentences]
    lst_groups = []
    group_idx = []

    for i in range(len(lst_counts)):
        if sum([lst_counts[k] for k in group_idx]) + lst_counts[i] <= max_len:
            group_idx.append(i)
        else:
            lst_groups.append(group_idx)

            j = 1
            while sum([lst_counts[k] for k in group_idx[-j:]]) < overlap:
                j += 1
            group_idx = group_idx[-j:] + [i]

    if len(group_idx) > 0:
        lst_groups.append(group_idx)

    lst_chunks = [[sentences[i] for i in g] for g in lst_groups]
    lst_chunks = [' \n '.join(i) for i in lst_chunks]
    return lst_chunks

def preprocess_phrase2_chunk_level(input_text):
    s = input_text
    s = re.sub('(?<=\n)[\n ]+', ' ', s)
    return s

# Chunking

In [13]:
lst_chunks = []

In [14]:
folders = os.listdir(data_path)

for folder in folders:
    files = os.listdir(data_path + "/" + folder)
    for file in files:
        if file.endswith(".txt"):
            with open(data_path + "/" + folder + "/" + file, 'r', encoding='utf-8') as f:
                content = f.read()
                content = preprocess_phrase1(content)
                chunks = markdown_chunking(content, MAX_LEN, OVERLAP)
                chunks = [preprocess_phrase2_chunk_level(c) for c in chunks]
                lst_chunks.append(chunks)

    break
        
        

In [21]:
pairs = []

In [31]:
def get_very_positive_pairs(lst_chunks):
    pairs = []
    for i in range(len(lst_chunks)):
        for j in range(i+1, len(lst_chunks)):
            pairs.append((lst_chunks[i], lst_chunks[j], 1))
    return pairs

In [33]:
def get_low_positive_pairs(lst_chunks1, lst_chunks2):
    pairs = []
    for i in range(len(lst_chunks1)):
        for j in range(len(lst_chunks2)):
            pairs.append((lst_chunks1[i], lst_chunks2[j], 0.5))
    return pairs

In [34]:
low_positives_pairs = get_low_positive_pairs(lst_chunks[0], lst_chunks[1])

In [38]:
len(lst_chunks[0])

9

In [36]:
len(low_positives_pairs)

99

In [ ]:
content = preprocess_phrase1(content)
out = markdown_chunking(content, max_len=MAX_LEN, overlap=OVERLAP)

# 